In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix, precision_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.neural_network import MLPClassifier
import streamlit as st

In [2]:
data = pd.read_csv('final_df.csv')

In [3]:
data.head()

,season,round,weather_warm,weather_cold,weather_dry,weather_wet,weather_cloudy,driver,grid,podium,...,constructor_minardi,constructor_prost,constructor_red_bull,constructor_renault,constructor_sauber,constructor_team_lotus,constructor_toro_rosso,constructor_toyota,constructor_tyrrell,constructor_williams
0,1983,1,False,False,True,False,False,keke_rosberg,1,15,...,0,0,0,0,0,0,0,0,0,1
1,1983,1,False,False,True,False,False,prost,2,6,...,0,0,0,1,0,0,0,0,0,0
2,1983,1,False,False,True,False,False,tambay,3,4,...,0,0,0,0,0,0,0,0,0,0
3,1983,1,False,False,True,False,False,piquet,4,1,...,0,0,0,0,0,0,0,0,0,0
4,1983,1,False,False,True,False,False,warwick,5,7,...,0,0,0,0,0,0,0,0,0,0


## Classification

In [4]:
df = data.copy()
df.podium = df.podium.map(lambda x: 1 if x == 1 else 0)

train = df[df.season <2026]
X_train = train.drop(['driver','podium'], axis = 1)
y_train = train.podium

scaler = StandardScaler()
X_train = pd.DataFrame(scaler.fit_transform(X_train), columns = X_train.columns)


In [5]:
def score_classification(model):
    score = 0
    for circuit in df[df.season == 2019]['round'].unique():

        test = df[(df.season == 2019) & (df['round'] == circuit)]
        X_test = test.drop(['driver','podium'], axis = 1)
        y_test = test.podium

        #scaling
        X_test = pd.DataFrame(scaler.transform(X_test), columns = X_test.columns)

        # make predictions
        prediction_df = pd.DataFrame(model.predict_proba(X_test), columns = ['proba_0', 'proba_1'])
        prediction_df['actual'] = y_test.reset_index(drop = True)
        prediction_df.sort_values('proba_1', ascending = False, inplace = True)
        prediction_df.reset_index(inplace = True, drop = True)
        prediction_df['predicted'] = prediction_df.index
        prediction_df['predicted'] = prediction_df.predicted.map(lambda x: 1 if x == 0 else 0)

        score += precision_score(prediction_df.actual, prediction_df.predicted)

    model_score = score / df[df.season == 2019]['round'].unique().max()
    
    return model_score

In [6]:
def race_winner_test(model,year,circuit):
    list_f = pd.DataFrame()
    test = df[(df.season == year) & (df['round'] == circuit)]
    X_test = test.drop(['driver','podium'], axis = 1)
    y_test = test.podium

        #scaling
    X_test = pd.DataFrame(scaler.transform(X_test), columns = X_test.columns)

        # make predictions
    list_f['driver'] = test['driver'].reset_index(drop = True)
    prediction_df = pd.DataFrame(model.predict_proba(X_test), columns = ['proba_0', 'proba_1'])
    prediction_df['actual'] = y_test.reset_index(drop = True)
    list_f['actual'] = prediction_df['actual']
    prediction_df.sort_values('proba_1', ascending = False, inplace = True)
    prediction_df.reset_index(inplace = True, drop = True)
    prediction_df['predicted'] = prediction_df.index
    prediction_df['predicted'] = prediction_df.predicted.map(lambda x: 1 if x == 0 else 0)
    list_f['predicted'] = prediction_df['predicted']
    list_f['Winning_prob'] = prediction_df['proba_1']
    list_f['Lossing_prob'] = prediction_df['proba_0']
    return list_f

In [7]:
comparison_dict ={'model':[],
                  'score': []}

In [8]:
model = MLPClassifier(hidden_layer_sizes = (75,25,50,10), activation = 'identity', 
                      solver = 'lbfgs', alpha = 1.6238e-02, random_state = 1)
model.fit(X_train, y_train)
model_score = score_classification(model)
comparison_dict['model'].append('neural_network_classifier')
comparison_dict['score'].append(model_score)

In [9]:
comparison_dict

{'model': ['neural_network_classifier'], 'score': [0.5714285714285714]}

In [10]:
season = int(input('Please enter the year'))
gp = int(input('enter the round number'))
final_model = race_winner_test(model,season,gp)
final_model

Please enter the year2021
enter the round number3


ValueError: Found array with 0 sample(s) (shape=(0, 98)) while a minimum of 1 is required by StandardScaler.